# Getting started with MIPLearn

## Introduction

**MIPLearn** is an open source framework that uses machine learning (ML) to accelerate the performance of both commercial and open source mixed-integer programming solvers (e.g. Gurobi, CPLEX, XPRESS, Cbc or SCIP). In this tutorial, we will:

1. Install the Python/Pyomo version of MIPLearn
2. Model a simple optimization problem using JuMP
3. Generate training data and train the ML models
4. Use the ML models together Gurobi to solve new instances

<div class="alert alert-info">
Note
    
The Python/Pyomo version of MIPLearn is currently only compatible with with Gurobi, CPLEX and XPRESS. For broader solver compatibility, see the Julia/JuMP version of the package.
</div>

<div class="alert alert-warning">
Warning
    
MIPLearn is still in early development stage. If run into any bugs or issues, please submit a bug report in our GitHub repository. Comments, suggestions and pull requests are also very welcome!
    
</div>


## Installation

MIPLearn is available in two versions:

- Python version, compatible with the Pyomo modeling language,
- Julia version, compatible with the JuMP modeling language.

In this tutorial, we will demonstrate how to use and install the Python/Pyomo version of the package. The first step is to install Python 3.8+ in your computer. See the [official Python website for more instructions](https://www.python.org/downloads/). After Python is installed, we proceed to install MIPLearn using `pip`:

In [1]:
!pip install MIPLearn==0.2.0.dev13

Defaulting to user installation because normal site-packages is not writeable


In addition to MIPLearn itself, we will also install Gurobi 9.1, a state-of-the-art commercial MILP solver. To succesfully complete this step, you will need a valid Gurobi license.

In [2]:
!pip install --upgrade -i https://pypi.gurobi.com gurobipy==9.1.2

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.gurobi.com


<div class="alert alert-info">
    
Note
    
In the code above, we install specific version of all packages to ensure that this tutorial keeps running in the future, even when newer (and possibly incompatible) versions of the packages are released. This is usually a recommended practice for all Python projects.
    
</div>

## Modeling a simple optimization problem

To illustrate how can MIPLearn be used, we will model and solve a small optimization problem related to power systems optimization. The problem we discuss below is a simplification of the **unit commitment problem,** a practical optimization problem solved daily by electric grid operators around the world. 

Suppose that you work at a utility company, and that it is your job to decide which electrical generators should be online at a certain hour of the day, as well as how much power should each generator produce. More specifically, assume that your company owns $n$ generators, denoted by $g_1, \ldots, g_n$. Each generator can either be online or offline. An online generator $g_i$ can produce between $p^\text{min}_i$ to $p^\text{max}_i$ megawatts of power, and it costs your company $c^\text{fix}_i + c^\text{var}_i y_i$, where $y_i$ is the amount of power produced. An offline generator produces nothing and costs nothing. You also know that the total amount of power to be produced needs to be exactly equal to the total demand $d$ (in megawatts). To minimize the costs to your company, which generators should be online, and how much power should they produce?

This simple problem can be modeled as a *mixed-integer linear optimization* problem as follows. For each generator $g_i$, let $x_i \in \{0,1\}$ be a decision variable indicating whether $g_i$ is online, and let $y_i \geq 0$ be a decision variable indicating how much power does $g_i$ produce. The problem is then given by:

$$
\begin{align}
\text{minimize } \quad & \sum_{i=1}^n \left( c^\text{fix}_i x_i + c^\text{var}_i y_i \right) \\
\text{subject to } \quad & y_i \leq p^\text{max}_i x_i & i=1,\ldots,n \\
& y_i \geq p^\text{min}_i x_i & i=1,\ldots,n \\
& \sum_{i=1}^n y_i = d \\
& x_i \in \{0,1\} & i=1,\ldots,n \\
& y_i \geq 0 & i=1,\ldots,n
\end{align}
$$

<div class="alert alert-info">
    
Note
    
We use a simplified version of the unit commitment problem in this tutorial just to make it easier to follow. MIPLearn can also handle realistic, large-scale versions of this problem. See benchmarks for more details.
    
</div>

Next, let us convert this abstract mathematical formulation into a concrete optimization model, using Python and Pyomo. We start by defining a class `UnitCommitmentInstance`, which holds all the input data. The class also contains a method `to_model` which constructs a concrete Pyomo model object:

In [12]:
import pyomo.environ as pe
import pyomo.opt as pyo
from miplearn import Instance

class UnitCommitmentInstance(Instance):
    def __init__(
        self,
        demand,
        pmin,
        pmax,
        cfix,
        cvar,
    ):
        super().__init__()
        self.demand = demand
        self.pmin = pmin
        self.pmax = pmax
        self.cfix = cfix
        self.cvar = cvar
        
    def to_model(self):
        n = len(self.pmin)
        model = pe.ConcreteModel()
        model.x = pe.Var(range(n), domain=pe.Binary)
        model.y = pe.Var(range(n), bounds=(0, float("inf")))
        model.obj = pe.Objective(
            expr=sum(
                self.cfix[i] * model.x[i] +
                self.cvar[i] * model.y[i]
                for i in range(n)
            )
        )
        model.eq_max_power = pe.ConstraintList()
        model.eq_min_power = pe.ConstraintList()
        for i in range(n):
            model.eq_max_power.add(model.y[i] <= self.pmax[i] * model.x[i])
            model.eq_min_power.add(model.y[i] >= self.pmin[i] * model.x[i])
        model.eq_demand = pe.Constraint(
            expr=sum(model.y[i] for i in range(n)) == self.demand,
        )
        return model

At this point, we can already use Pyomo and any mixed-integer linear programming solver to find optimal solutions to any instance of this problem. To illustrate this, let us solve a small instance with three generators:

In [14]:
instance = UnitCommitmentInstance(
    demand = 100.0,
    pmin = [10, 20, 30],
    pmax = [50, 60, 70],
    cfix = [700, 600, 500],
    cvar = [1.5, 2.0, 2.5],
)
model = instance.to_model()
opt = pyo.SolverFactory("gurobi")
opt.solve(model)

print("obj = ", pe.value(model.obj))
print("  x = ", [pe.value(model.x[i]) for i in range(3)])
print("  y = ", [pe.value(model.y[i]) for i in range(3)])


obj =  1320.0
  x =  [0.0, 1.0, 1.0]
  y =  [0.0, 60.0, 40.0]


Running the code above, we found that the optimal solution for our small problem instance costs \$1320. It is achieve by keeping generators 2 and 3 online and producing, respectively, 60 MW and 40 MW of power.

## Generating training data

Although Gurobi could solve the small example above in a fraction of a second, it gets slower for larger and more complex versions of the problem. If this is a problem that needs to be solved frequently, as it is often the case in practice, it could make sense to spend some time upfront generating a **trained** version of Gurobi, which can solve new instances (similar to the ones it was trained on) faster.

In the following, we will use MIPLearn to train machine learning models that can be used to accelerate Gurobi's performance on a particular set of instances. More specifically, MIPLearn will train a model that is able to predict the optimal solution for instances that follow a given probability distribution, then it will provide this predicted solution to Gurobi as a warm start.

Before we can train the model, we need to collect training data by solving a large number of instances. In real-world situations, we may construct these training instances based on historical data. In this tutorial, we will construct them using a random instance generator:

In [42]:
from scipy.stats import uniform
import random

def random_uc_instances(samples, n, seed=42):
    random.seed(seed)
    pmin = uniform(loc=100_000.0, scale=400_000.0).rvs(n)
    pmax = pmin * uniform(loc=2.0, scale=2.5).rvs(n)
    cfix = pmin * uniform(loc=100.0, scale=25.0).rvs(n)
    cvar = uniform(loc=1.25, scale=0.25).rvs(n)
    return [
        UnitCommitmentInstance(
            demand = pmax.sum() * uniform(loc=0.5, scale=0.25).rvs(),
            pmin = pmin,
            pmax = pmax,
            cfix = cfix,
            cvar = cvar,
        )
        for i in range(samples)
    ]

In this example, for simplicity, only the demands change from one instance to the next. We could also have made the prices and the production limits random. The more randomization we have in the training data, however, the more challenging it is for the machine learning models to learn solution patterns.

Now we generate 100 instances of this problem, each one with 1,000 generators. We will use the first 90 instances for training, and the remaining 10 instances to evaluate performance.

In [43]:
instances = random_uc_instances(samples=100, n=1000);
train_instances = instances[1:90]
test_instances = instances[91:100];

model = instances[0].to_model()
opt = pyo.SolverFactory("gurobi")
opt.solve(model, tee=True);

Using license file /home/axavier/gurobi.lic
Using gurobi.env file
Set parameter Threads to value 1
Read LP format model from file /tmp/tmpl37dgdq_.pyomo.lp
Reading time = 0.01 seconds
x2001: 2002 rows, 2001 columns, 5001 nonzeros
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (linux64)
Thread count: 16 physical cores, 32 logical processors, using up to 1 threads
Optimize a model with 2002 rows, 2001 columns and 5001 nonzeros
Model fingerprint: 0x637c283e
Variable types: 1001 continuous, 1000 integer (1000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+06]
  Objective range  [1e+00, 6e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+08]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 2001 rows, 2000 columns, 5000 nonzeros
Variable types: 1000 continuous, 1000 integer (1000 binary)

Root relaxation: objective 2.143251e+10, 1010 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl 

Next, we write these objects to individual files. MIPLearn uses files during the training process because, for large-scale optimization problems, it is often impractical to hold the entire training data, as well as the concrete Pyomo models, in memory. Files also make it much easier to solve multiple instances simultaneously, potentially even on multiple machines. We will cover parallel and distributed computing in a future tutorial.

The code below generates the files `uc/train/00001.h5`, `uc/train/00002.h5`, etc., which contain the input data in HDF5 format.

In [7]:
from miplearn.instance.file import FileInstance
from glob import glob
import os

os.makedirs("uc/train", exist_ok=True)
os.makedirs("uc/test", exist_ok=True)

for (i, instance) in enumerate(train_instances):
    FileInstance.save(instance, f"uc/train/{i:05d}.h5")
    
for (i, instance) in enumerate(test_instances):
    FileInstance.save(instance, f"uc/test/{i:05d}.h5")

train_instances = [FileInstance(f) for f in glob("uc/train/*.h5")]
test_instances = [FileInstance(f) for f in glob("uc/test/*.h5")]

Finally, we use `LearningSolver` to solve all the training instances. `LearningSolver` is the main component provided by MIPLearn, which integrates MIP solvers and ML. The solutions, and other useful training data, are stored in the same HDF5 files.

In [8]:
from miplearn import LearningSolver
from tqdm import tqdm

solver = LearningSolver()
for instance in tqdm(train_instances):
    solver.solve(instance)

 12%|█▏        | 11/89 [00:50<05:56,  4.57s/it]


KeyboardInterrupt: 

## Solving new instances

With training data in hand, we can now fit the ML models, using the `LearningSolver.fit` method, then solve the test instances with `MIPLearn.solve!`, as shown below:

In [ ]:
solver_ml = LearningSolver()
solver_ml.fit(train_instances)
for instance in tqdm(test_instances):
    solver_ml.solve(instance)

The trained MIP solver was able to solve all test instances in about 6 seconds. To see that ML is being helpful here, let us repeat the code above, but remove the `fit!` line:

In [ ]:
solver_baseline = LearningSolver()
for instance in tqdm(test_instances):
    solver_baseline.solve(instance)

Without the help of the ML models, SCIP took around 10 seconds to solve the same test instances.

<div class="alert alert-info">
Note
    
Note that is is not necessary to specify what ML models to use. MIPLearn, by default, will try a number of classical ML models and will choose the one that performs the best, based on k-fold cross validation. MIPLearn is also able to automatically collect features based on the MIP formulation of the problem and the solution to the LP relaxation, among other things, so it does not require handcrafted features. If you do want to customize the models and features, however, that is also possible, as we will see in a later tutorial.
</div>

## Understanding the acceleration

Let us go a bit deeper and try to understand how exactly did MIPLearn accelerate Gurobi's performance. First, we are going to solve one of the test instances again, using the trained solver, but this time using the `tee=True` parameter, so that we can see Gurobi's log:

In [ ]:
solver_ml.solve(test_instances[0], tee=True);

The log above is quite complicated if you have never seen it before, but the important line is the one starting with `feasible solution found [...] objective value 1.705169e+07`. This line indicates that MIPLearn was able to construct a warm start with value `1.705169e+07`. Using this warm start, SCIP then used the branch-and-cut method to either prove its optimality or to find an even better solution. Very quickly, however, SCIP proved that the solution produced by MIPLearn was indeed optimal. It was able to do this without generating a single cutting plane or running any other heuristics; it could tell the optimality by the root LP relaxation alone, which was very fast. 

Let us now repeat the process, but using the untrained solver this time:

In [ ]:
solver_baseline.solve(test_instances[0], tee=True);

In this log file, notice how the previous line about warm starts is missing. Since no warm starts were provided, SCIP had to find an initial solution using its own internal heuristics, which are not specifically tailored for this problem. The initial solution found by SCIP's heuristics has value `2.335200e+07`, which is significantly worse than the one constructed by MIPLearn. SCIP then proceeded to improve this solution, by generating cutting planes and repeatedly running additional primal heuristics. In the end, it was able to find the optimal solution, as expected, but it took longer.

In summary, MIPLearn accelerated the solution process by constructing a high-quality initial solution. In the following tutorials, we will see other strategies that MIPLearn can use to accelerate MIP performance, besides warm starts.

## Accessing the solution

In the example above, we used `MIPLearn.solve!` together with data files to solve both the training and the test instances. The solutions were saved to a `.h5` files in the train/test folders, and could be retrieved by reading theses files, but that is not very convenient. In this section we will use an easier method.

We can use the function `MIPLearn.load!` to obtain a regular JuMP model:

In [ ]:
model = MIPLearn.load("uc/test/000001.jld2", build_uc_model)

We can then solve this model as before, with `MIPLearn.solve!`:

In [ ]:
solve!(solver_ml, model)
println("obj = ", objective_value(model))
println("  x = ", round.(value.(model[:x][1:10])))
println("  y = ", round.(value.(model[:y][1:10]), digits=2))